In [3]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
from pdfminer.layout import LTCurve, LTLine, LTTextContainer, LTChar
from pdfminer.high_level import extract_text_to_fp
import pandas as pd
import math
import numpy as np

In [4]:
b = []
for p in extract_pages("all.pdf"):
    b.append(p)

C:\Users\kurud\Anaconda3\lib\site-packages\pdfminer\pdfpage.py:159: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='all.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


In [19]:
curvetypes = ["arrow head", "horizntalV"]


def get_my_key(obj):
      return obj[1]

def fillcurveTypes(curve):
    for i, j in curves.iterrows():
        prop = (vars(j.element))
        pts = prop["pts"]
        ptslen = len(pts)
        if ptslen == 4:
            curves.iat[i,1]=curvetypes[0]
        else:
            curves.iat[i,1]=curvetypes[1]
            
            
def findaxesdistance(pts1,pts2,index):
    out = -1
    if pts1[index] > pts2[index]:
        out = min(pts1[index] - pts2[2+index],pts1[2+index] - pts2[2+index])
    else:
        out = min(pts2[index] - pts1[2+index],pts2[2+index] - pts1[2+index])
    return out if out>0 else 0

def finddistance(pts1,pts2):
    xmin = findaxesdistance(pts1,pts2,0)
    ymin = findaxesdistance(pts1,pts2,1)
    dist = math.sqrt((xmin**2)+(ymin**2))
    return dist

def getdistances(vc,curves):
    heads = []
    vcprop = (vars(vc.element))
    vcpts = vcprop["bbox"]
    dist = []
    for i, j in curves.iterrows():
        ctype = (j.type)
        if ctype == curvetypes[0]:
            prop = (vars(j.element))
            pts = prop["bbox"]
            dist.append((j.element,finddistance(pts,vcpts)))
    return dist

def findvcurveheads(vc,curves):
    dist = getdistances(vc,curves)
    dist.sort(key=get_my_key)
#     print(dist)
    return [i for (i,j) in dist[0:2]] 

def fillVcurvesheads(curves):
    for i, j in curves.iterrows():
        ctype = (j.type)
        if ctype == curvetypes[1]:
            curves.iat[i,2] = findvcurveheads(j,curves)

            
def findpointboxaxisdis(pt,box,index):
    out = -1
    if box[index] > pt[index]:
        out = min(box[index] - pt[index],box[2+index] - pt[index])
    else:
        out = min(-box[index] + pt[index],-box[2+index] + pt[index])
    return out if out>0 else 0

def getdisbtwptbox(pt,box):
    xmin = findpointboxaxisdis(pt,box,0)
    ymin = findpointboxaxisdis(pt,box,1)
    dist = math.sqrt((xmin**2)+(ymin**2))
    return dist


def getdistanceslnheads(vc,curves):
    heads = []
    vcprop = (vars(vc.element))
    vcpts = vcprop["pts"]
    dist = []
    for i, j in curves.iterrows():
        if vc.element != j.element:
            prop = (vars(j.element))
            box = prop["bbox"]
            mindis = 1000
            for pt in vcpts:
                d = getdisbtwptbox(pt,box)
                if d< mindis:
                    mindis=d
            dist.append((j.element,mindis))
    return dist
    
    

def fillLinessheads(lines,curves):
    for i, j in lines.iterrows():
        dist = getdistanceslnheads(j,curves)
        dist.sort(key=get_my_key)
#         print(dist)
        if dist[0][1]<=0:
            lines.iat[i,3] = dist[0][0]            

def gettail(vcpts,box):
    maxd = -1
    tailpt = None
    for pt in vcpts:
        d = getdisbtwptbox(pt,box)
        if d >maxd:
            maxd = d
            tailpt = pt
    return tailpt

def getdistanceslntails(vc,lines):
    heads = []
    vcprop = (vars(vc.element))
    vcpts = vcprop["pts"]
    headbox = vars(vc.headcurve)["bbox"]
    tailpt = gettail(vcpts,headbox)
    dist = []
#     print(tailpt)
    for i, j in lines.iterrows():
        if vc.element != j.element:
            prop = (vars(j.element))
            box = prop["bbox"]
            d = getdisbtwptbox(tailpt,box)
            dist.append((j.element,d))
    return dist
    
def ishorizontal(element):
    prop = vars(element)
    box = prop["bbox"]
    return (box[1]-box[3])==0

def fillLinetails(lines):
    for i, j in lines.iterrows():
        
        if isinstance(j.headcurve, LTCurve) & ishorizontal(j.element):
            dist = getdistanceslntails(j,lines)
            dist.sort(key=get_my_key)
#             print(j.element)
#             print(dist[0])
            if dist[0][1]<3:
                lines.iat[i,1] = dist[0][0] 
            

def iarrowsverticallyallinged(vbox,jbox):
    out = False
    if (jbox[1] <= vbox[1]) & (jbox[1] <= vbox[3]):
            if (jbox[3] >= vbox[1]) & (jbox[3] >= vbox[3]):
                out = True
    return out

def getdistances1(vc,lines):
    heads = []
    vcprop = (vars(vc.headcurve))
    vbox = vcprop["bbox"]
    dist = []
    for i, j in lines.iterrows():
        if not ishorizontal(j.element):
            prop = (vars(j.element))
            jbox = prop["bbox"]
            xd = jbox[0]-vbox[2]
            if iarrowsverticallyallinged(vbox,jbox) & ((xd>=0)|((xd>-1)&(xd<0))):
                dist.append((j.element,abs(xd)))
    return dist

def fillLineheadLines(lines):
    for i, j in lines.iterrows():
        
        if isinstance(j.headcurve, LTCurve) & ishorizontal(j.element):
            dist = getdistances1(j,lines)
            dist.sort(key=get_my_key)
            if len(dist)>0:
#                 print(j.element)
#                 print(dist[0])
                if dist[0][1]<5:
                    lines.iat[i,2] = dist[0][0] 
            
texttypes = ["treatment","steps","header","footer"]


def getyrange(texts):
    ymin = 5000
    ymax = 0
    for i, j in textboxes.iterrows():
        if j.element.get_text() == 'Version 2.2021, 07/19/2021 © 2021 National Comprehensive Cancer Network® (NCCN®), All rights reserved. NCCN Guidelines® and this illustration may not be reproduced in any form without the express written permission of NCCN.\n':
            ymin = vars(j.element)["bbox"][3]
        elif j.element.get_text() == 'NCCN Guidelines Version 2.2021\nAcute Lymphoblastic Leukemia\n':
            ymax = vars(j.element)["bbox"][1]
    return ymin, ymax

def isheaderbox(element,YMINPAGE,YMAXPAGE):
    txtymin = vars(element)["bbox"][1]
    return txtymin >= YMAXPAGE

def isfooterbox(element,YMINPAGE,YMAXPAGE):
    txtymax = vars(element)["bbox"][3]
    return txtymax <= YMINPAGE

def gettopverdis(txtelement,jelement):
    tbox = vars(txtelement)["bbox"]
    jbox = vars(jelement)["bbox"]
    return 5000 if (jbox[1] - tbox[3]) <0 else (jbox[1] - tbox[3])

def getlefthordis(txtelement,jelement,YMINPAGE,YMAXPAGE):
    tbox = vars(txtelement)["bbox"]
    jbox = vars(jelement)["bbox"]
    out = tbox[0]
    if (tbox[0] >= jbox[0]) & (tbox[1]>jbox[3])  & (not isfooterbox(jelement,YMINPAGE,YMAXPAGE)):
        if tbox[0] <= jbox[2]:
            out = jbox[0]
    return out
    
def gettype(txt,texts,YMINPAGE,YMAXPAGE):
    topbox = None
    ymin = 5000
    typebox = None
    txtymin = vars(txt.element)["bbox"][1]
    for i, j in texts.iterrows():
        if j.element != txt.element:
            dis = gettopverdis(txt.element,j.element)
            if ymin > dis:
                topbox = j.element
                ymin = dis
    
    if isheaderbox(txt.element,YMINPAGE,YMAXPAGE):
        typebox = texttypes[2]
    elif isfooterbox(txt.element,YMINPAGE,YMAXPAGE):
        typebox = texttypes[3]
    elif isheaderbox(topbox,YMINPAGE,YMAXPAGE):
        typebox = texttypes[0]
    else: 
        typebox = texttypes[1]
    
    xmin = 5000
    if typebox == texttypes[0]:
        for i, j in texts.iterrows():
            if j.element != txt.element:
                dis = getlefthordis(txt.element,j.element,YMINPAGE,YMAXPAGE)
                if xmin > dis:
                    topbox = j.element
                    xmin = dis
    
    return typebox, xmin
            

def filltextboxestypes(texts,YMINPAGE,YMAXPAGE):
     for i, j in texts.iterrows():
        ttype, xleftmargin = gettype(j,texts,YMINPAGE,YMAXPAGE)
        texts.iat[i,1] = ttype
        texts.iat[i,2] = xleftmargin

In [20]:
textboxes = pd.DataFrame(columns=['element','type','leftmargin','verleftline','verrightline','lines','statename','nextsubstate','nextstate'])
lines = pd.DataFrame(columns=['element','tailHorEle','headHorEle','headcurve'])
curves = pd.DataFrame(columns=['element','type','headcurve'])
txtbxlist = []
linelist  = []
curvelist = []
for element in b[12]:
        if isinstance(element, LTTextContainer):
#             print(element)
#             print(element.get_text())
            txtbxlist.append(element)
        elif isinstance(element, LTLine): 
            linelist.append(element)
        elif isinstance(element, LTCurve):
            curvelist.append(element)
textboxes['element'] = txtbxlist
lines['element'] = linelist
curves['element'] = curvelist

In [21]:
fillcurveTypes(curves)

In [22]:
fillVcurvesheads(curves)

In [23]:
fillLinessheads(lines,curves)

In [24]:
lines[lines['headcurve'].notna()].count()

element       14
tailHorEle     0
headHorEle     0
headcurve     14
dtype: int64

In [25]:
lines1 = lines.copy()
fillLinetails(lines1)

In [26]:
lines1[lines1['tailHorEle'].notna()].count()

element       8
tailHorEle    8
headHorEle    0
headcurve     8
dtype: int64

In [27]:
lines2 = lines1.copy()
fillLineheadLines(lines2)

In [28]:
lines2[lines2['headHorEle'].notna()].count()

element       9
tailHorEle    7
headHorEle    9
headcurve     9
dtype: int64

In [29]:
textboxes1 = textboxes.copy()
YMINPAGE,YMAXPAGE = getyrange(textboxes)
filltextboxestypes(textboxes1,YMINPAGE,YMAXPAGE)

In [30]:
textboxes1[textboxes1['type']==texttypes[0]]

,element,type,leftmargin,verleftline,verrightline,lines,statename,nextsubstate,nextstate
3,"((<LTChar 24.912,501.165,32.854,512.165 matrix...",treatment,24.7191,NaN,NaN,NaN,NaN,NaN,NaN
4,"((<LTChar 162.048,501.063,168.769,512.063 matr...",treatment,161.8551,NaN,NaN,NaN,NaN,NaN,NaN
5,"((<LTChar 490.528,501.063,498.470,512.063 matr...",treatment,486.9539,NaN,NaN,NaN,NaN,NaN,NaN


In [669]:
for row in textboxes1[textboxes1['type']==texttypes[0]]:
    print(row)

element
type
leftmargin
verleftline
verrightline
lines
statename
nextsubstate
nextstate


In [675]:
def gettreatmenttype(element,texts):
    ebox = vars(element)['bbox']
    typeTr = None
    for i, row in texts[texts['type']==texttypes[0]].iterrows():
        lm  = row.leftmargin
        if lm <= ebox[0]:
            typeTr = row.element.get_text()
    return typeTr
        
def getpointTextbox(pt,texts):
    dist = []
    for i, j in texts.iterrows():
        prop = (vars(j.element))
        box = prop["bbox"]
        d = getdisbtwptbox(pt,box)
        dist.append((j.element,d))
    
    dist.sort(key=get_my_key)
    
    return dist[0][0]
    
def getheadboxtailpt(line):
    head = line.headcurve
    prop = (vars(line.element))
    pts = prop["pts"]
    headbox = vars(head)["bbox"]
    tailpt = gettail(pts,headbox)
    return tailpt, headbox

def getboxsTextbox(headbox,texts):
    dist = []
    for i, j in texts.iterrows():
        prop = (vars(j.element))
        box = prop["bbox"]
        d = finddistance(headbox,box)
        dist.append((j.element,d))
        
    dist.sort(key=get_my_key)
    
    return dist[0][0]

def gettexttypebybox(headbox, texts):
    headtxtele = getboxsTextbox(headbox,texts)
    
    headtxt = headtxtele.get_text()
    
    headtype = gettreatmenttype(headtxtele,texts)
    
    return headtxt, headtype

def gettexttypebypoint(tailpt, texts):
    
    tailtxtele = getpointTextbox(tailpt,texts)
    
    tailtxt = tailtxtele.get_text()
    
    tailtype = gettreatmenttype(tailtxtele,texts)
    
    return tailtxt, tailtype

def getrealtionptbox(tailpt, headbox, texts):
    
    tailtxt, tailtype, headtxt, headtype = (None,None,None,None)
#     print(tailpt)    
    tailtxt, tailtype = gettexttypebypoint(tailpt,texts)
    
    headtxt, headtype = gettexttypebybox(headbox,texts)
    
    return (tailtxt, tailtype, headtxt, headtype)
    

def gettextcase1(line,texts):
    tailpt, headbox = getheadboxtailpt(line)    
    return getrealtionptbox(tailpt, headbox, texts)

def gettextcase2(line,texts):
    tailpt, headbox = getheadboxtailpt(line)
    headbox = vars(line.headHorEle)['bbox']
    return getrealtionptbox(tailpt, headbox, texts)

def getrealtionboxbox(tailbox, headbox, texts):
    tailtxt, tailtype, headtxt, headtype = (None,None,None,None)
#     print(tailpt)    
    tailtxt, tailtype = gettexttypebybox(tailbox,texts)
    
    headtxt, headtype = gettexttypebybox(headbox,texts)
    
    return (tailtxt, tailtype, headtxt, headtype)

def gettextcase3(line,texts):
    tailpt, headbox = getheadboxtailpt(line) 
    tailbox = vars(line.tailHorEle)['bbox']
    return getrealtionptbox(tailbox, headbox, texts)


def gettextcase4(line,texts):
    tailbox = vars(line.tailHorEle)['bbox']
    headbox = vars(line.headHorEle)['bbox']
    return getrealtionptbox(tailbox, headbox, texts)


def gettext(line,texts):
    
    if (line.tailHorEle is np.nan) & (line.headHorEle is np.nan):
        return gettextcase1(line,texts)
    elif (line.tailHorEle is np.nan) & (line.headHorEle is not np.nan):
        return gettextcase2(line,texts)
    elif (line.tailHorEle is not np.nan) & (line.headHorEle is np.nan):
        return gettextcase3(line,texts)
    elif (line.tailHorEle is not np.nan) & (line.headHorEle is not np.nan):
        return gettextcase4(line,texts)


def findrelations(lines,texts):
    relations = []
    for i,j in lines.iterrows():
        if j.headcurve is not np.nan:
            tailtxt, tailtype, headtxt, headtype = gettext(j,texts)
            relations.append((tailtxt, tailtype, headtxt, headtype))
    return relations

In [677]:
relations = findrelations(lines2, textboxes1)

In [678]:
relations

[('AYAn,p,q and \nadult patients \n<65 years of \nageo without \nsubstantial \ncomorbidities\n',
  'RISK \nSTRATIFICATIONh,i\n',
  'Clinical trial \nor\nTyrosine kinase \ninhibitor (TKI)  + \nchemotherapyt\nor\nTKI + \ncorticosteroidt\n',
  'TREATMENT INDUCTIONff,r,s\n'),
 ('Adult patients \n≥65 years \nof ageo,bb \nor with \nsubstantial \ncomorbidities \n',
  'RISK \nSTRATIFICATIONh,i\n',
  'Clinical trial \nor\nTKI + \ncorticosteroidt,cc\nor\nTKI + \nchemotherapyt,cc\n',
  'TREATMENT INDUCTIONff,r,s\n'),
 ('CR\n',
  'TREATMENT INDUCTIONff,r,s\n',
  'Monitoring for \nMRDu,v\nSee MRD \nAssessment \n(ALL-F)\n',
  'TREATMENT INDUCTIONff,r,s\n'),
 ('Less than CR\n',
  'TREATMENT INDUCTIONff,r,s\n',
  'See Relapsed/Refractory \nDisease (ALL-7)\n',
  'CONSOLIDATION THERAPYff,r,s\n'),
 ('Persistent/\nRising \nMRDdd,ee\n',
  'TREATMENT INDUCTIONff,r,s\n',
  'Blinatumomab ± TKIt,ee \n(B-ALL)gg,hh\nor\nContinue multiagent \nchemotherapy/\ncorticosteroid + TKIt,ee\nor\nAllogeneic hematopoietic \

In [690]:
with open("relations.csv",'w',encoding='utf-8') as f:
    for e in relations:
        for i in e:
            f.write('"{}",'.format(i).replace('\n','\\n'))
        f.write('\n')

In [36]:
for element in b[40]:
    print(element)
    print(vars(element))
    if hasattr(element,'get_text') :
        print(element.get_text())

<LTTextBoxHorizontal(0) 15.000,600.551,616.573,607.551 'Printed by Dinesh KURUBA on 12/5/2021 1:32:46 PM. For personal use only. Not approved for distribution. Copyright © 2021 National Comprehensive Cancer Network, Inc., All Rights Reserved.\n'>
{'x0': 15.0, 'y0': 600.551, 'x1': 616.5730000000004, 'y1': 607.551, 'width': 601.5730000000004, 'height': 7.0, 'bbox': (15.0, 600.551, 616.5730000000004, 607.551), '_objs': [<LTTextLineHorizontal 15.000,600.551,616.573,607.551 'Printed by Dinesh KURUBA on 12/5/2021 1:32:46 PM. For personal use only. Not approved for distribution. Copyright © 2021 National Comprehensive Cancer Network, Inc., All Rights Reserved.\n'>], 'index': 0}
Printed by Dinesh KURUBA on 12/5/2021 1:32:46 PM. For personal use only. Not approved for distribution. Copyright © 2021 National Comprehensive Cancer Network, Inc., All Rights Reserved.

<LTTextBoxHorizontal(1) 186.617,542.950,465.716,582.550 'NCCN Guidelines Version 2.2021\nAcute Lymphoblastic Leukemia\n'>
{'x0': 186